## crop images and annotations

In [1]:
import os
import cv2

# Parameters
input_folder = "input_images"  # Folder containing images and annotations
output_folder = "output_images"  # Folder to save cropped images and annotations
crop_width = 640
crop_height = 640

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Global variables for mouse callback
crop_start = None
fixed_crop_box = None


# Mouse callback function
def draw_fixed_rectangle(event, x, y, flags, param):
    global crop_start, fixed_crop_box

    if event == cv2.EVENT_LBUTTONDOWN:
        # When the left mouse button is clicked, set the starting point for the crop
        crop_start = (x, y)

    elif event == cv2.EVENT_LBUTTONUP and crop_start:
        # Finalize the fixed rectangle on mouse release
        x1, y1 = crop_start
        x2 = x1 + crop_width
        y2 = y1 + crop_height

        # Ensure the rectangle doesn't exceed the image boundaries
        img_height, img_width = param.shape[:2]
        x2 = min(x2, img_width)
        y2 = min(y2, img_height)
        x1 = max(0, x1)
        y1 = max(0, y1)

        fixed_crop_box = (x1, y1, x2, y2)


# Function to process a single image and its annotation
def process_image(image_path, annotation_path, output_image_path, output_annotation_path):
    global fixed_crop_box

    # Open the image with OpenCV
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]
    img_copy = img.copy()

    # Display the image and allow the user to draw the fixed crop box
    cv2.imshow("Draw Fixed Crop Box", img)
    cv2.setMouseCallback("Draw Fixed Crop Box", draw_fixed_rectangle, param=img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Ensure a crop box was defined
    if not fixed_crop_box:
        print(f"Skipping {image_path}: No crop box defined.")
        return

    # Crop the image
    x1, y1, x2, y2 = fixed_crop_box
    cropped_img = img_copy[y1:y2, x1:x2]
    cv2.imwrite(output_image_path, cropped_img)

    # Adjust YOLO annotations
    if os.path.exists(annotation_path):
        with open(annotation_path, "r") as file:
            lines = file.readlines()

        new_annotations = []
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())

            # Convert normalized to absolute coordinates
            x_center_abs = x_center * img_width
            y_center_abs = y_center * img_height
            width_abs = width * img_width
            height_abs = height * img_height

            # Skip annotations outside the crop box
            if x_center_abs < x1 or x_center_abs > x2 or y_center_abs < y1 or y_center_abs > y2:
                continue

            # Adjust coordinates for cropping
            x_center_abs -= x1
            y_center_abs -= y1

            # Convert back to normalized coordinates for the cropped size
            new_img_width = x2 - x1
            new_img_height = y2 - y1
            x_center_new = x_center_abs / new_img_width
            y_center_new = y_center_abs / new_img_height
            width_new = width_abs / new_img_width
            height_new = height_abs / new_img_height

            # Save the adjusted annotation
            new_annotations.append(f"{int(class_id)} {x_center_new:.6f} {y_center_new:.6f} {width_new:.6f} {height_new:.6f}\n")

        # Write the new annotations to a file
        with open(output_annotation_path, "w") as file:
            file.writelines(new_annotations)


# Loop through all images in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # File paths
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, os.path.splitext(filename)[0] + ".txt")
        output_image_path = os.path.join(output_folder, filename)
        output_annotation_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".txt")

        # Process the image and annotation
        process_image(image_path, annotation_path, output_image_path, output_annotation_path)

print("All images and annotations processed successfully!")


FileNotFoundError: [Errno 2] No such file or directory: 'input_images'

## Crop movie without annotations

In [7]:
import cv2

def crop_video(input_path, output_path):
    # Open the video
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame to select ROI
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Let the user draw a rectangular ROI on the first frame
    # selectROI returns (x, y, w, h)
    r = cv2.selectROI("Select Crop Region", frame, fromCenter=False, showCrosshair=True)
    cv2.destroyWindow("Select Crop Region")

    x, y, w, h = r

    # Get video parameters
    fps = cap.get(cv2.CAP_PROP_FPS)
    # FourCC is a 4-byte code used to specify the video codec
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # or 'XVID', 'AVC1', etc.

    # Create a VideoWriter to write the cropped frames
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    # Process the already-read first frame
    cropped_first_frame = frame[y:y+h, x:x+w]
    out.write(cropped_first_frame)

    # Loop over the rest of the frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cropped_frame = frame[y:y+h, x:x+w]
        out.write(cropped_frame)

    # Release everything
    cap.release()
    out.release()
    print(f"Cropped video saved at: {output_path}")

if __name__ == "__main__":
    input_video = "/home/bohbot/Evyatar/exp/Joni_showoff/nectar/joni.mp4"
    output_video = "/home/bohbot/Evyatar/exp/Joni_showoff/nectar/joni_640.mp4"
    crop_video(input_video, output_video)


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Cropped video saved at: /home/bohbot/Evyatar/exp/Joni_showoff/nectar/joni_640.mp4


In [ ]:
/home/bohbot/Evyatar/runs/detect/BFv112/weights/best.pt